In [ ]:
# Extract from https://github.com/StanfordVL/behavior/blob/main/behavior/benchmark/behavior_benchmark.py
import os
import numpy as np

from PIL import Image
from igibson.utils.utils import parse_config

from igibson import object_states
from igibson.utils.utils import restoreState

from igibson.action_primitives.fetch_robot_semantic_actions_env import FetchRobotSemanticActionEnv
from igibson.custom_utils import get_env_config, print_properties
from igibson.primitives_utils import get_names_of_visible_obj_inside, get_objects_inside, settle_physics 
import igibson.render_utils as render_utils

import warnings
warnings.filterwarnings("ignore")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Env config
task = "cleaning_out_drawers"
scene_id =  "Benevolence_1_int"

# Init env
sim_env = FetchRobotSemanticActionEnv(task, scene_id, verbose=False)

In [ ]:
def print_symbolic_state(state):
    print("-"*76)
    print("Symbolic state: \n")
    for k in state:
        true_states = {}
        for key in state[k].keys():
            val = state[k][key] # bool
            if val: 
                true_states[key] = val
                
        print(k, true_states)
    print("-"*76)

def execute_plan(sim_env, plan):
    image, symbolic_state = sim_env.get_state_and_image()

    image.show()
    print_symbolic_state(symbolic_state)
    
    for action in plan:
        try:
            success, image, symbolic_state = sim_env.apply_action(action)
            print(f'Action {action} executed. Success: {success}')

            image.show()
            print_symbolic_state(symbolic_state)
            
            legal = True
            
        except (ValueError, AssertionError) as e:
            print(f"Action {action} failed: {e}")
            success = False
            legal = False

# Plan 1: Place on top of sink

In [ ]:
plan = [
    {'action':'navigate-to', 'params':['cabinet_1']},
    {'action':'open-container', 'params':['cabinet_1']},
    {'action':'navigate-to', 'params':['cabinet_1']}, # not needed, but useful to stress-test 
    {'action':'close-container', 'params':['cabinet_1']}
]

In [ ]:
plan = [
    {'action':'navigate-to', 'params':['cabinet_1']},
    {'action':'open-container', 'params':['cabinet_1']}, 
    {'action':'grasp', 'params':['bowl_1']}, 
    {'action':'navigate-to', 'params':['sink_1']},
    {'action':'place-on', 'params':['bowl_1','sink_1']},
    {'action':'navigate-to', 'params':['cabinet_1']},
    {'action':'grasp', 'params':['bowl_2']},
    {'action':'navigate-to', 'params':['sink_1']},
    {'action':'place-next-to', 'params':['bowl_2','bowl_1']},
    ]

In [ ]:
image, symbolic_state = sim_env.get_state_and_image()

image.show()
print_symbolic_state(symbolic_state)

for action in plan:
    try:
        success, image, symbolic_state = sim_env.apply_action(action)
        print(f'Action {action} executed. Success: {success}')

        image.show()
        print_symbolic_state(symbolic_state)
        
        legal = True
        
    except (ValueError, AssertionError) as e:
        print(f"Action {action} failed: {e}")
        success = False
        legal = False

# Plan 2: Place inside cabinet

In [ ]:
plan = [
    {'action':'grasp', 'params':{'obj_name':'bowl.n.01_1'}},
    {'action':'go_to', 'params':{'obj_name':'cabinet.n.01_1'}},
    {'action':'place_inside', 'params':{'trg_obj_name':'bowl.n.01_1', 'container_obj_name':'cabinet.n.01_1'}},
]

In [ ]:
execute_plan(sim_env, plan)